<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Diego Bartolucci
- Nombre de alumno 2: Pilar Nilo

### **Link de repositorio de GitHub:** [Repositorio](https://github.com/DiegoBarto01/MDS7202-pili-barto.git)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [ ]:
!pip install -qq xgboost optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.8 MB/s eta 0:00:00


# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/MyDrive/Lab_9'
except:
    print('Ignorando conexión drive-colab')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv(f'{path}/sales.csv')

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


In [ ]:
df['date'] = pd.to_datetime(df['date'])
# df.dtypes

<ipython-input-4-4a129d44759f>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])


,0
id,int64
date,datetime64[ns]
city,object
lat,float64
long,float64
pop,int64
shop,object
brand,object
container,object
capacity,object


## 1. Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [ ]:
from sklearn import set_config
set_config(transform_output="pandas")
# 1
# Se separa la columna que se desea predecir la cual corresponde a quantity
X = df.drop(columns=['quantity'])
y = df['quantity']

# Se separan los datos de entrenamiento, validación y test
from sklearn.model_selection import train_test_split
X_temporal, X_test, y_temporal, y_test = train_test_split(X, y, test_size=0.1, random_state=444)
X_train, X_val, y_train, y_val = train_test_split(X_temporal, y_temporal, test_size=0.222, random_state=444)

In [ ]:
# 2
# Se implementa un FunctionTransformer para extraer el día, mes y año de la variable date. Se guardan estas variables en el formato categorical de pandas
from sklearn.preprocessing import FunctionTransformer
def fecha_date(df):
    df['day'] = df['date'].dt.day.astype('category')
    df['month'] = df['date'].dt.month.astype('category')
    df['year'] = df['date'].dt.year.astype('category')
    return df

In [ ]:
#3 Implemente un ColumnTransformer para procesar de manera adecuada los datos numéricos y categóricos.
# Use OneHotEncoder para las variables categóricas. Nota: Utilice el método .set_output(transform='pandas') para obtener un DataFrame como salida del ColumnTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
var_categoricas=['city','shop','brand','container','capacity']
var_minmaxScaler=['price','lat','long']
var_date=['date']
var_non=['pop']

#Se crea un pipeline_date para que no existan problemas posteriores con el tipo de la columna
pipeline_date = Pipeline(steps=[('Parse_date',FunctionTransformer(fecha_date)),
                                ('OneHotEncoder', OneHotEncoder(sparse_output=False))
                                ])

column_transformer=ColumnTransformer([('MinMax Scaler', MinMaxScaler(), var_minmaxScaler),
                                      ('One Hot Encoder', OneHotEncoder(sparse_output=False), var_categoricas),
                                      ('parse_Date', pipeline_date, var_date),
                                      ('Non', 'passthrough', var_non)
                                      ]).set_output(transform='pandas')



In [ ]:
column_transformer

ColumnTransformer(transformers=[('MinMax Scaler', MinMaxScaler(),
                                 ['price', 'lat', 'long']),
                                ('One Hot Encoder',
                                 OneHotEncoder(sparse_output=False),
                                 ['city', 'shop', 'brand', 'container',
                                  'capacity']),
                                ('parse_Date',
                                 Pipeline(steps=[('Parse_date',
                                                  FunctionTransformer(func=<function fecha_date at 0x789a42aea170>)),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(sparse_output=False))]),
                                 ['date']),
                                ('Non', 'passthrough', ['pop'])])

In [ ]:
#4
#Guarde los pasos anteriores en un Pipeline, dejando como último paso el regresor DummyRegressor para generar predicciones en base a promedios

from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
pipeline_dummy = Pipeline(steps=[('preprocessor', column_transformer),
                                 ('Dummy_reg', DummyRegressor(strategy='mean'))])


In [ ]:
#5
#Entrene el pipeline anterior y reporte la métrica mean_absolute_error sobre los datos de validación
pipeline_dummy.fit(X_train, y_train)
y_pred_dummy = pipeline_dummy.predict(X_val)
mae_dummy = mean_absolute_error(y_val, y_pred_dummy)

In [ ]:
mae_dummy

13289.38914448816

¿Cómo se interpreta esta métrica para el contexto del negocio?

Interpretando este valor en el contexto del negocio, correspondería a que la predicción se encuentra con un error de 13000 unidades aproximadamente de su valor real. Indicando que es un mal modelo para predecir las cantidades requeridas.

In [ ]:
#6
#Vuelva a entrenar el Pipeline pero esta vez usando XGBRegressor como modelo utilizando los parámetros por default.
pipeline_xgb = Pipeline(steps=[('preprocessor', column_transformer),
                               ('regressor', XGBRegressor(enable_categorical=True))])
pipeline_xgb.fit(X_train, y_train)
y_pred_xgb = pipeline_xgb.predict(X_val)
mae_xgb = mean_absolute_error(y_val, y_pred_xgb)

In [ ]:
mae_xgb

2641.753636712196

¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el DummyRegressor?

Se puede observar una disminución notable en el MAE del algoritmo disminuyendo de 13000 a 2600. Este modelo presenta una clara mejora en relacion al dummyregressor anteriormente utilizado mostrando valores menores de MAE.

In [ ]:
#7 SOLO USAR ANTES DE ENTREGAR PARA NO TENER 1000 pkls
#Guarde ambos modelos en un archivo .pkl
import joblib
joblib.dump(pipeline_dummy, f'{path}/pipeline_dummy.pkl')
joblib.dump(pipeline_xgb, f'{path}/pipeline_xgb.pkl')

['/content/drive/MyDrive/Lab_9/pipeline_xgb.pkl']

## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [ ]:
#1
#volver a entrenar el pipeline con XGBRegressor, forzando una relacion monotona negativa entre precio y cantidad
monotone_constraint = {'MinMax Scaler__price': -1}
pipeline_xgb_forced = Pipeline(steps=[('preprocessor', column_transformer),
                               ('regressor', XGBRegressor(enable_categorical=True, monotone_constraints=monotone_constraint))])
#entreno pipeline
pipeline_xgb_forced.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('MinMax Scaler',
                                                  MinMaxScaler(),
                                                  ['price', 'lat', 'long']),
                                                 ('One Hot Encoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['city', 'shop', 'brand',
                                                   'container', 'capacity']),
                                                 ('parse_Date',
                                                  Pipeline(steps=[('Parse_date',
                                                                   FunctionTransformer(func=<function fecha_date at 0x789a42aea170>)),
                                                                  ('OneHotEncoder',
                                                                   OneHot...
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints={'MinMax Scaler__price': -1},
                              multi_strategy=None, n_estimators=None,
                              n_jobs=None, num_parallel_tree=None,
                              random_state=None, ...))])

In [ ]:
#2
#volver a reportar el MAE sobre el conjunto de validacion
pipeline_xgb_forced.fit(X_train, y_train)
y_pred_xgb_forced = pipeline_xgb_forced.predict(X_val)
mae_xgb_forced = mean_absolute_error(y_val, y_pred_xgb_forced)

In [ ]:
mae_xgb_forced

2761.511911072187

*3. ¿Como cambia el error al incluir esta relacion? ¿tenia razon su amigo?*

Al incluir la relación se puede observar un mínimo cambio en el MAE obtenido por lo que se puede concluir que la relación entre el precio y la cantidad no es tan influyente.

In [ ]:
#4. Guardar modelo en archivo .pkl
joblib.dump(pipeline_xgb_forced, f'{path}/pipeline_xgb_forced.pkl')

['/content/drive/MyDrive/Lab_9/pipeline_xgb_forced.pkl']

## 3. Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

In [ ]:
import optuna
from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)
# Inserte su código acá

def objective(trial):
    #minimizar el MAE en el conjunto de validacion, usando el metodo .set_user_attr() para almacenar el mejor pipeline entrenado
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)

    pipeline_date = Pipeline(steps=[('Parse_date',FunctionTransformer(fecha_date)),
                                ('OneHotEncoder', OneHotEncoder(sparse_output=False, min_frequency=min_frequency))
                                ])

    #Definir los hiperparámetros a optimizar
    xgbr_regressor_hiperparametros={
        'learning_rate' : trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators' : trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight' : trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0, 1)
        }

    column_transformer=ColumnTransformer([('MinMax Scaler', MinMaxScaler(), var_minmaxScaler),
                                      ('One Hot Encoder', OneHotEncoder(sparse_output=False,min_frequency=min_frequency), var_categoricas),
                                      ('parse_Date', pipeline_date,var_date),
                                      ('Non', 'passthrough', var_non)
                                      ])

    pipeline_xgb_optuna = Pipeline([
                                ('preprocessor', column_transformer),
                               ('xgb_regressor', XGBRegressor(**xgbr_regressor_hiperparametros,
                                                              random_state=444))
                               ])
    pipeline_xgb_optuna.fit(X_train, y_train)
    y_pred_xgb_optuna = pipeline_xgb_optuna.predict(X_val)
    mae_xgb_optuna = mean_absolute_error(y_val, y_pred_xgb_optuna)
    trial.set_user_attr('best_pipeline', pipeline_xgb_optuna)

    return mae_xgb_optuna


In [ ]:
#2 fijar el entrenamiento a 5 minutos
study = optuna.create_study(sampler=TPESampler(seed=444), direction='minimize')
study.optimize(objective, timeout=300, show_progress_bar=True) #5*60=300

   0%|          | 00:00/05:00

In [ ]:
#3 Optimizar el modelo y reportar el número de trials, el MAE y los mejores hiperparámetros encontrados.
mejores_hiperparametros = study.best_params
mejor_mae=study.best_value
print(f'Mejores hiperparámetros: {mejores_hiperparametros}')
print(f'Mejor MAE: {mejor_mae}')
print(f'Número de Trials:{len(study.trials)}')

Mejores hiperparámetros: {'min_frequency': 0.048349072305207894, 'learning_rate': 0.08078802482020431, 'n_estimators': 994, 'max_depth': 9, 'max_leaves': 91, 'min_child_weight': 5, 'reg_alpha': 0.06243463735328504, 'reg_lambda': 0.817349509662139}
Mejor MAE: 2092.3875013108222
Número de Trials:164


*4.Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?*



1.   Learning_rate: Es la tasa de aprendizaje, es decir que va a determinar la rapidez con la que aprende el modelo. Entre más baja la tasa, más lento aprende el modelo y el valor de 0.08 no es tan baja dentro del rango dado por ende el modelo podría considerarse como de aprendizaje rápido.
2.   n_estimators: Son los números de árboles que se construyen, entre más alto su valor, más eficiente.  
3.  max_depth: Es la profundidad máxima de cada árbol, entre más profundo más complejo, pero esto podría provocar sobreajustes.
4. max_leaves: Es el número máximo de hojas en cada árbol, funciona similar a max_depth
5. min_child_weight: Controla la cantidad mínima de instancias de muestra necesarias en un nodo hoja. Entre más bajo su valor, se cpturan relaciones más detalladas y en forma opuesta, entre más alto el valor, más robusto se vuelve el modelo.
6. reg_alpha: Controla la magnitud absoluta de los pesos de las hojas, este parametro permite eliminar caracteristicas irrelevantes, lo que ayuda a prevenir el sobreajuste.
7. reg_lambda: Funciona similar a reg_alpha, pero se diferencia en que evita que el modelo se generalice.
8. min_frequency: Permite filtrar caracteristicas que aparecen con menos frecuencia.


Los rangos de optimización indicados nos parecen acordes, pero genera ruido que max_leaves, reg_alpha, reg_lambda y min_frequecy puedan ser 0 ya que esto implica que ciertas caracteristicas no tengan frecuencia y se eliminen, que el modelo se generalize y que no se eliminen carcateristicas irrelevantes (esto podría deteriorar las métricas al ser entrenado el modelo ya que tendrá caracteristicas que no suman al entrenamiento).
       






In [ ]:
mejores_hiperparametros['min_frequency']

0.048349072305207894

In [ ]:
#redefino los hiperparametros para optimizar
min_frequency_best=mejores_hiperparametros['min_frequency']
xgb_mejores_hiperparametros = {

'learning_rate':mejores_hiperparametros['learning_rate'],
'n_estimators':mejores_hiperparametros['n_estimators'],
'max_depth':mejores_hiperparametros['max_depth'],
'max_leaves':mejores_hiperparametros['max_leaves'],
'min_child_weight':mejores_hiperparametros['min_child_weight'],
'reg_alpha':mejores_hiperparametros['reg_alpha'],
'reg_lambda':mejores_hiperparametros['reg_lambda']}
#redefino los pipelines en base a lo anterior
pipeline_date_best=Pipeline(steps=[('Parse_date',FunctionTransformer(fecha_date)),
                                ('OneHotEncoder', OneHotEncoder(sparse_output=False, min_frequency=min_frequency_best))
                                ])
column_transformer_best=ColumnTransformer([('MinMax Scaler', MinMaxScaler(), var_minmaxScaler),
                                      ('One Hot Encoder', OneHotEncoder(sparse_output=False,min_frequency=min_frequency_best), var_categoricas),
                                      ('parse_Date', pipeline_date_best, var_date),
                                      ('Non', 'passthrough', var_non)
                                      ])
pipeline_xgb_optuna_best=Pipeline([
                                ('preprocessor', column_transformer_best),
                               ('xgb_regressor', XGBRegressor(**xgb_mejores_hiperparametros,
                                                              random_state=444))
                               ])
#fit
pipeline_xgb_optuna_best.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('MinMax Scaler',
                                                  MinMaxScaler(),
                                                  ['price', 'lat', 'long']),
                                                 ('One Hot Encoder',
                                                  OneHotEncoder(min_frequency=0.048349072305207894,
                                                                sparse_output=False),
                                                  ['city', 'shop', 'brand',
                                                   'container', 'capacity']),
                                                 ('parse_Date',
                                                  Pipeline(steps=[('Parse_date',
                                                                   FunctionTransformer(func=<function fecha_date at 0x789a...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None,
                              learning_rate=0.08078802482020431, max_bin=None,
                              max_cat_threshold=None, max_cat_to_onehot=None,
                              max_delta_step=None, max_depth=9, max_leaves=91,
                              min_child_weight=5, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=994, n_jobs=None,
                              num_parallel_tree=None, random_state=444, ...))])

In [ ]:
#5 guardar en pkl
#import joblib
joblib.dump(pipeline_xgb_optuna_best, f'{path}/pipeline_xgb_optuna_best.pkl')

['/content/drive/MyDrive/Lab_9/pipeline_xgb_optuna_best.pkl']

## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [ ]:
!pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 2.6 MB/s eta 0:00:00


1. El prunning corresponde a una técnica utilizada para la optimización de hiperparámetros y reducir el tiempo de entrenamiento al descartar ensayos poco viables en las etapas iniciales. En lugar de terminar el entrenamiento de todos los ensayos, revisa los resultados parciales y detiene aquellos que no están rindiendo lo suficiente, permitiendo concentrar los recursos en las configuraciones con más potencial.

In [ ]:
# import optuna
from optuna.integration import XGBoostPruningCallback
import xgboost as xgb
# from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Se redefine la función objetive
def objective(trial):
  #Se definen los hiperparametros para XGBoost

  learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
  n_estimators = trial.suggest_int('n_estimators', 50, 1000)
  max_depth = trial.suggest_int('max_depth', 3, 10)
  max_leaves = trial.suggest_int('max_leaves', 0, 100)
  min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
  reg_alpha = trial.suggest_float('reg_alpha', 0, 1)
  reg_lambda = trial.suggest_float('reg_lambda', 0, 1)


  #Se define el hiperparametro para OneHotEncoder
  min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)

  column_transformer=ColumnTransformer([('MinMax Scaler', MinMaxScaler(), var_minmaxScaler),
                                      ('One Hot Encoder', OneHotEncoder(sparse_output=False,min_frequency=min_frequency), var_categoricas),
                                      ('parse_Date', pipeline_date,var_date),
                                      ('Non', 'passthrough', var_non)
                                      ])

  #Se preprocesan los datos para XGBoost
  X_train_pruning = column_transformer.fit_transform(X_train)
  X_val_pruning = column_transformer.transform(X_val)

  #Se preparan los datos para poder utilizarlos en XGBoost
  dtrain = xgb.DMatrix(X_train_pruning, label=y_train)
  dval = xgb.DMatrix(X_val_pruning, label=y_val)

  #Se agrega el pruningcallback
  pruning_callback = XGBoostPruningCallback(trial, "validation-mae")

  #Se entrena el modelo de XGboost
  evals = [(dtrain, "train"), (dval, "validation")]
  model_params = {
      'objective': 'reg:squarederror',
      'eval_metric': 'mae',
      'learning_rate': learning_rate,
      'max_depth': max_depth,
      'max_leaves': max_leaves,
      'min_child_weight': min_child_weight,
      'reg_alpha': reg_alpha,
      'reg_lambda': reg_lambda,
      'verbosity': 0
  }

  model_pruning = xgb.train(
      params=model_params,
      dtrain=dtrain,
      num_boost_round=n_estimators,
      evals=evals,
      early_stopping_rounds=10,
      callbacks=[pruning_callback],
      verbose_eval=False #Para que no escupa 1000 líneas de código
  )

  #Se predice, se calcula el mae y se almacena lo mejor
  y_pred_xgb_optuna_pruning = model_pruning.predict(dval)
  mae_xgb_optuna_pruning = mean_absolute_error(y_val, y_pred_xgb_optuna_pruning)
  trial.set_user_attr('best_model_pruning', model_pruning)

  return mae_xgb_optuna_pruning

In [ ]:
study = optuna.create_study(sampler=TPESampler(seed=444), direction='minimize')
study.optimize(objective, timeout=300, show_progress_bar=True)

   0%|          | 00:00/05:00

In [ ]:
mejores_hiperparametros_pruning = study.best_params
mejor_mae_pruning = study.best_value
print(f'Mejores hiperparámetros: {mejores_hiperparametros_pruning}')
print(f'Mejor MAE: {mejor_mae_pruning}')
print(f'Número de Trials:{len(study.trials)}')

Mejores hiperparámetros: {'learning_rate': 0.0938704435508794, 'n_estimators': 266, 'max_depth': 10, 'max_leaves': 92, 'min_child_weight': 5, 'reg_alpha': 0.6022040358139161, 'reg_lambda': 0.08671045032237236, 'min_frequency': 0.16873246588167323}
Mejor MAE: 2282.389185389576
Número de Trials:281


In [ ]:
study.best_trial.user_attrs["best_model_pruning"]

Se puede analizar que se realizaron mayores cantidades de Trials para ajustarse de mejor manera a los parámetros utilizados. A pesar de esto no se obtuvieron mejores resultados a la sección anterior. Esto se puede deber a el funcionamiento del pruning, ya que este proceso puede acortar trials si no ve un buen funcionamiento de este por lo que podría estar eliminando trials que finalmente tendrían buenos resultados.

In [ ]:
# Guardar el modelo en un archivo .pkl
best_model_pruning = study.best_trial.user_attrs["best_model_pruning"]
joblib.dump(best_model_pruning, f'{path}/pruning_model.pkl')

['/content/drive/MyDrive/Lab_9/pruning_model.pkl']

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [ ]:
# Inserte su código acá
import optuna.visualization as vis
#historial de optimización
fig1 = vis.plot_optimization_history(study)
fig1.show()

Se puede observar que desde el trial nº7 se generan mejoras considerables en el MAE generado.

In [ ]:
#coordenadas paralelas
fig2 = vis.plot_parallel_coordinate(study)
fig2.show()

Se pueden observar los valores máximos para max_depths(10), max_leaves(100) y min_child_weight(5) indicando que mientras se presente un mayor valor, mejores resultados se obtienen, al igual que con n_estimators. Además se ven valores altos para reg_alpha y reg_alpha pero se puede observar que no tienen un gran impacto general en el valor objetivo. Finalmente se puede observar que con una menor min_frecuency se obtienen mejores resultados objetivos además de mantener un valor de learning_rate no muy alto (para evitar subajsute) y muy bajo (para evitar sobreajuste).

In [ ]:
#importancia parametros
fig3 = vis.plot_param_importances(study)
fig3.show()

Se puede ver claramente que los hiperparámetros que más importancia tienen en el modelo corresponden al learning_rate utilizado por lo antes mencionado y la min_frecuency ya que este mientras menor sea se presentaban resultados óptimos.

## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

In [ ]:
#1. genere una tabla resumen del MAE en el conjunto de validacion obtenido en los 5 modelos entrenados desde baseline hasta XGBoost con constrains, optune y prunning
df_resume_mae=pd.DataFrame(columns=['Modelo','MAE'])
#relleno el df
df_resume_mae.loc[0]=['Baseline',mae_dummy]
df_resume_mae.loc[1]=['XGBoost',mae_xgb]
df_resume_mae.loc[2]=['XGBoost con Constraints',mae_xgb_forced]
df_resume_mae.loc[3]=['XGBoost con Optuna',mejor_mae]
df_resume_mae.loc[4]=['XGBoost con Prunning',mejor_mae_pruning]
df_resume_mae

,Modelo,MAE
0,Baseline,13289.389144
1,XGBoost,2641.753637
2,XGBoost con Constraints,2761.511911
3,XGBoost con Optuna,2092.387501
4,XGBoost con Prunning,2282.389185


*2. Comparación de resultados y mejor rendimiento*

El mejor rendimiento lo obtiene el modelo XGBoost con Optuna dado que posee el menor valor de MAE y entre menor sea este valor, menos errores posee el modelo al momento de predecir.

El método que usa Prunning (esté se encarga de podar) podría hacer pensar que sería el mejor modelo al realizar podaciones, pero Optuna obtiene los mejores hiperparametros para lograr la mejor métrica entre todos los modelos.

In [ ]:
print(path +'/pipeline_xgb_optuna_best.pkl')

/content/drive/MyDrive/Lab_9/pipeline_xgb_optuna_best.pkl


In [ ]:
#3. cargar el mejor modelo, predecir sobre el conjunto de test y reportar MAE
mejor_modelo_mae=joblib.load(path +'/pipeline_xgb_optuna_best.pkl')
test_pred=mejor_modelo_mae.predict(X_test)
mae_test=mean_absolute_error(y_test, test_pred)
print(f'MAE en el conjunto de test: {mae_test}')

MAE en el conjunto de test: 2068.5054309834745


*4. Existen diferencias respecto a las metricas obtenidas en conjunto val? porque ocurre?*

Si existen diferencias respecto a las métricas obtenidas en el conjunto de validación, pero esta es leve. Incluso es mejor la métrica obtenida en el conjunto test que la obtenida en el conjunto de validación. Esta pequeña diferencia se puede estar produciendo por el tamaño entre los datasets (validación, entrenamiento y test), siendo el test el más pequeño de los tres y también estaría afectando la distribución que tiene cada uno de los sets.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>